In [ ]:
import clingo
import re
from json import dumps

In [ ]:
def combine(encoding_path, benchmark_path, destination_path):
    
    combined_path = destination_path + "/combined.lp"
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [ ]:
def save_plans(plans, k=0):
    if type(plans)==list:
        for i in range(len(plans)):
            if type(plans[i])==str:
                with open("./python/solution_%d_%d.lp" % (k,i), 'w') as f:
                    f.write(plans[i])
            else:
                if type(plans[i])==list:
                    k+=1
                    save_plans(plans[i],k)
    else:
        if type(plans)==str:
                with open("./python/solution_%d.lp" % (k), 'w') as f:
                    f.write(plans)

In [ ]:
def solve(robot_id, destination_path, restrictions=[], boundary=int()):
    if boundary<=4:
        boundary=5
    combined_path = destination_path + "/combined.lp"
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    bound = "#const boundary = {}.".format(boundary)
    for i in range(5,100):
        solution = ""
        horizon = "#const horizon = {}.".format(i)
        asp = horizon + "\n" + rid + "\n" + bound + "\n" + combined
        if restrictions:
            for j in restrictions:
                if j:
                    asp = asp + "\n" + j
            
        ctl = clingo.Control(['--stats'])
        ctl.add("base", [], asp)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle: solution=("{} ".format(m))
            handle.get()
        print(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': ')))
        solution = solution.replace(' ', '. ')
        if solution:
            return solution, i
            break

In [ ]:
def prep(robots, destination, restrictions=[]):
    plans=[]
    plan_=""
    paths = []
    actions = []
    
    if not paths:
        for i in robots:
            paths.append([])
    
    for i in robots:
        temp = solve(i, destination, restrictions)
        paths[i-1].append(temp[0])
        actions.append(temp[1])
        if paths[i-1][0]:
            plan_ = plan_ + paths[i-1][0]
    return paths, plan_, actions

In [ ]:
def collisions(robot_id, encoding_path, plan):
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    asp = rid + "\n" + plan + "\n" + encoding
    solution_ = ""
    li = []
    
    ctl = clingo.Control(['--stats'])
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: 
            solution_=("{} ".format(m))
            li = re.findall('s\((.+?),',("{} ".format(m)))
        handle.get()
    print(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': ')))
    li=list(set(map(int,li)))
    if solution_== " ":
        return
    else:
        solution_ = solution_.replace(' ', '. ')
        return solution_, li

In [ ]:
def plan(robots, destination, collision, paths, plan_, actions, boundary, restrictions=[], actions_=[]):
    if not actions_ or sum(actions_)==0:
        actions_ = list(actions)
    
    max_value = max(actions_)
    max_index = actions_.index(max_value)
    cur_rest = []
    temp_ = collisions(actions_[max_index], collision, plan_)
    if temp_:
        cur_rest.append(temp_[0])
    actions_[max_index] = 0
    
    if cur_rest:
        restrictions.append(cur_rest[-1])
        restricted_robots = temp_[1]
        for i in restricted_robots:
            temp = solve(i, destination, restrictions, boundary)
            paths[i-1] = [temp[0]]
            actions[i-1] = temp[1]
        plan_ = ""
        for i in robots:
            if paths[i-1][0]:
                plan_ = plan_ + paths[i-1][0]
        plan_ = plan(robots, destination, collision, paths, plan_, actions, boundary, restrictions, actions_)
    return plan_

In [ ]:
enc = "./asprilo-encodings/shortest_path/pathfinder.lp"
ben = "./instances/example4/instance.lp"
dest = "./python"
col = "./asprilo-encodings/collision/collision.lp"
robots = []
for i in range(1,26):
    robots.append(i)

rest=[]
act=[]
plans =[]


combine(enc, ben, dest)

In [ ]:
start = time.time()
paths, plan_, actions = prep(robots, dest, rest)
boundary=min(actions)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
plans_=plan(robots, dest, col, paths, plan_, actions, boundary, rest, act)
end = time.time()
save_plans(plans_)
print(end-start)